In [4]:
import sys  
from PyQt4.QtGui import *  
from PyQt4.QtCore import *  
from PyQt4.QtWebKit import * 
import requests
import json
import re
import pickle

url = 'http://www.sephora.com/eye-cream-dark-circles?pageSize=-1'

class Render(QWebPage):
    def __init__(self, url):
    self.app = QApplication(sys.argv)
    QWebPage.__init__(self)
    self.loadFinished.connect(self._loadFinished)
    self.mainFrame().load(QUrl(url))
    self.app.exec_()
  
  def _loadFinished(self, result):
    self.frame = self.mainFrame()
    self.app.quit()

r = Render(url)
html = r.frame.toHtml()


products = unicode(re.search(r"(?<=products).*?(?=meta)", html).group(0)).replace('}}],"', '}}').replace('":[{', '{')

p = products.split('},')

prods = []

for product in p[:-1]:
    prods.append(product + '}')
    
prods.append(p[-1])

products = {}

for prod in prods:
    product = json.loads(prod)
    key = product['product_url']
    products[key] = {
        'rating': product['rating'],
        'brand': product['brand_name'],
        'id': product['id'],
        'name': product['display_name']
    }

url_base = 'http://www.sephora.com'

for product in products.keys():
    url = url_base + product
    response = requests.get(url)
    page = response.text
    try:
        reviews = int(unicode(re.search(r"(?<=\n).*?(?= review)", page).group(0)).strip())
    except ValueError:
        reviews = 0
    products[product]['num_reviews'] = reviews

with open('sephora_data.pkl', 'w') as picklefile:
    pickle.dump(products, picklefile)

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 20)

In [5]:
import scrapy

class SephoraPerfumeSpider(scrapy.Spider): # Create sephora perfume subclass of Spider class provided by Scrapy
    name = 'sephora_perfume'
    # start_urls = ['https://www.sephora.com/shop/perfume?currentPage=%' % page for page in range(1,14)] # Range should be one more than you want
    start_urls = ['https://www.sephora.com/shop/perfume']
    # What we want to do is get the HTML content and then get pull data out of tags
    # For this we're going to want the brand, the price, the name of the perfume
    # We'll also want the link to the page to get the description
    def parse(self, response):
        # To get attributes we want, define selectors
        name_selector = 'span[data-at="sku_item_name"]::text'
        brand_selector = 'span[data-at="sku_item_brand"]::text'
        price_selector = 'span[data-at="sku_item_price_list"]::text'
        product_names = response.css(name_selector).getall()
        product_brands = response.css(brand_selector).getall()
        product_prices = response.css(price_selector).getall()

        for index,product in enumerate(product_names):
            yield{
            'product_name': product_names[index],
            'brand_name': product_brands[index],
            'product_price': product_prices[index]
        }

In [6]:
import requests
from bs4 import BeautifulSoup, NavigableString, Tag
import re
from datetime import datetime
import pandas as pd
import psycopg2
import time



page = requests.get('https://www.sephora.com/shop/perfume?currentPage=1')

# Create a BeautifulSoup object
soup = BeautifulSoup(page.text, 'html.parser')

# Can be illustrative to look at data directly
soup

spans = soup.find_all('span')